In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re

In [2]:
df_train = pd.read_excel("../Raw_data/Data_Train.xlsx")
df_test = pd.read_excel("../Raw_data/Data_Test.xlsx")

In [3]:
df_full = df_train.append(df_test)

/home/sidhartha/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
df_full['Restaurant_name'] = df_full.Restaurant.apply(lambda x: x.split("_")[1])

In [5]:
location_encode = LabelEncoder()

In [6]:
df_full['transformed_location'] = location_encode.fit_transform(df_full.Location)

In [7]:
city = {"FTI College, Law College Road, Pune":"Pune",
 "Sector 3, Marathalli":"Bangalore",
 "Mumbai Central":"Mumbai",
"Sector 1, Noida":"Delhi",
 "Rmz Centennial, I Gate, Whitefield":"Bangalore",
 "Delhi University-GTB Nagar":"Delhi",
"Yerawada, Pune, Maharashtra":"Pune",
 "Delhi Administration Flats, Timarpur":"Delhi",
 "Moulali, Kolkata":"Kolkata",
"Dockyard Road, Mumbai CST Area":"Mumbai",
 "Pune University":"Pune",
 "Gora Bazar, Rajbari, North Dumdum, Kolkata":"Kolkata",
"D-Block, Sector 63, Noida":"Delhi",
 "Sector 14, Noida":"Delhi",
 "Mico Layout, Stage 2, BTM Layout,Bangalore":"Bangalore",
"Laxman Vihar Industrial Area, Sector 3A, Gurgoan":"Gurgoan",
 "Tiretti, Kolkata":"Kolkata",
 "Sandhurst Road, Mumbai CST Area":"Mumbai",
 "MG Road, Pune":"Pune",
 "Hyderabad Public School, Begumpet":"Hyderabad",
"Majestic":"Bangalore",
 "Chandni Chowk, Kolkata":"Kolkata",
 "Delhi High Court, India Gate":"Delhi",
"Chatta Bazaar, Malakpet, Hyderabad":"Hyderabad",
 "Sector 63A,Gurgaon":"Gurgaon",
 "Delhi Cantt.":"Delhi",
"Tejas Nagar Colony, Wadala West, Mumbai":"Mumbai",
 "Babarpur, New Delhi, Delhi":"Delhi",
"Nathan Road, Mangaldas Road, Pune":"Pune",
 "Panjetan Colony, Malakpet, Hyderabad":"Hyderabad",
"Raja Bazar, Kolkata":"Kolkata",
 "Jaya Nagar, Saidabad, Hyderabad":"Hyderabad",
 "Noorkhan Bazaar, Malakpet, Hyderabad":"Hyderabad",
"Musi Nagar, Malakpet, Hyderabad":"Hyderabad",
 "BTM Layout 1, Electronic City":"Bangalore"}


In [8]:
df_full['City'] = df_full.Location.map(city)
city_enc =LabelEncoder()
df_full['City'] = city_enc.fit_transform(df_full.City)
df_full['Count_cuisins'] = df_full.Cuisines.apply(lambda x: len(x.split(",")))

In [9]:
df_full.head()

,Average_Cost,Cuisines,Delivery_Time,Location,Minimum_Order,Rating,Restaurant,Reviews,Votes,Restaurant_name,transformed_location,City,Count_cuisins
0,₹200,"Fast Food, Rolls, Burger, Salad, Wraps",30 minutes,"FTI College, Law College Road, Pune",₹50,3.5,ID_6321,4,12,6321,10,7,5
1,₹100,"Ice Cream, Desserts",30 minutes,"Sector 3, Marathalli",₹50,3.5,ID_2882,4,11,2882,30,0,2
2,₹150,"Italian, Street Food, Fast Food",65 minutes,Mumbai Central,₹50,3.6,ID_1595,30,99,1595,19,6,3
3,₹250,"Mughlai, North Indian, Chinese",30 minutes,"Sector 1, Noida",₹99,3.7,ID_5929,95,176,5929,28,1,3
4,₹200,"Cafe, Beverages",65 minutes,"Rmz Centennial, I Gate, Whitefield",₹99,3.2,ID_6123,235,521,6123,26,0,2


In [10]:
cuisines = []
for i in df_full.Cuisines.values:
    try:
        cuisines.append(i.split(","))
    except:
        cuisines.append(i)

In [11]:
cuisines_cleaned = []
for i in cuisines:
    cuisines_cleaned+=i

In [12]:
final_cuisins = []
for i in list(set(cuisines_cleaned)):
    text = re.findall(r'\w+',i)
    text = (" ").join(text)
    final_cuisins.append(text)

In [13]:
map_dict = dict(zip(set(final_cuisins),[i for i in range (len(set(final_cuisins)))]))

In [14]:
# pd.DataFrame({"cuisins":list(set(final_cuisins))}).to_csv("test.csv")

In [15]:
df_time_taken = pd.read_csv("test.csv")

In [16]:
time = dict(zip(df_time_taken.cuisins,df_time_taken.time_to_cook))

In [17]:
def cuisine_clear(cuisine):
    global map_dict
    cuisines = []
    try:
        cuisines.append(cuisine.split(","))
    except:
        cuisines.append(cuisine)
    
    cuisines_cleaned = []
    for i in cuisines:
        cuisines_cleaned+=i
    final_cuisins = []
    for i in list(set(cuisines_cleaned)):
        text = re.findall(r'\w+',i)
        text = (" ").join(text)
        for i,k in  map_dict.items():
            if i == text:
                final_cuisins.append(k)
    return sum(final_cuisins)

In [18]:
df_full['Cuisines_enc'] = df_full.Cuisines.apply(cuisine_clear)

In [19]:
def average_time_taken(cuisine):
    global time
    cuisines = []
    try:
        cuisines.append(cuisine.split(","))
    except:
        cuisines.append(cuisine)
    
    cuisines_cleaned = []
    for i in cuisines:
        cuisines_cleaned+=i
    final_cuisins = []
    for i in list(set(cuisines_cleaned)):
        text = re.findall(r'\w+',i)
        text = (" ").join(text)
        for i,k in  time.items():
            if i == text:
                final_cuisins.append(k)
    return int(sum(final_cuisins)/len(final_cuisins))

In [20]:
# df_full["Average_cooking_time"] = df_full.Cuisines.apply(average_time_taken)

In [21]:
# from sklearn.multiclass import ClassifierMixin

In [22]:
import re
df_full = df_full[df_full['Average_Cost']!='for']
df_full['Average_Cost'] = df_full['Average_Cost'].apply(lambda x: re.findall(r"\d+",x)[0])

In [23]:
df_full['Minimum_Order'] = df_full['Minimum_Order'].apply(lambda x: re.findall(r"\d+",x)[0])

In [24]:
df_full.head()

,Average_Cost,Cuisines,Delivery_Time,Location,Minimum_Order,Rating,Restaurant,Reviews,Votes,Restaurant_name,transformed_location,City,Count_cuisins,Cuisines_enc
0,200,"Fast Food, Rolls, Burger, Salad, Wraps",30 minutes,"FTI College, Law College Road, Pune",50,3.5,ID_6321,4,12,6321,10,7,5,324
1,100,"Ice Cream, Desserts",30 minutes,"Sector 3, Marathalli",50,3.5,ID_2882,4,11,2882,30,0,2,90
2,150,"Italian, Street Food, Fast Food",65 minutes,Mumbai Central,50,3.6,ID_1595,30,99,1595,19,6,3,186
3,250,"Mughlai, North Indian, Chinese",30 minutes,"Sector 1, Noida",99,3.7,ID_5929,95,176,5929,28,1,3,74
4,200,"Cafe, Beverages",65 minutes,"Rmz Centennial, I Gate, Whitefield",99,3.2,ID_6123,235,521,6123,26,0,2,96


In [25]:
df_full = df_full.reset_index(drop = True)

In [26]:
val = ['-','NEW','Opening Soon','Temporarily Closed']
val1 = ['-']
val2 = ['Opening Soon','Temporarily Closed']

df_full.Rating = df_full.Rating.apply(lambda x:np.nan if x in val else float(x) )
def missing_value_rating(df):
    df.Rating = df.Rating.fillna(df.Rating.mean())
    return df

df_full = df_full.groupby("transformed_location").apply(missing_value_rating)
df_full.Rating = df_full.Rating.fillna(df_full.Rating.mean())


def missing_votes(df):
    df.Votes = df.Votes.fillna(df.Votes.mean()) 
    return df

df_full.Votes = df_full.Votes.apply(lambda x:np.nan if x=='-' else float(x) )
# # df_full.Votes = df_full.Votes.fillna(int(df_full.Votes.mean()))

df_full =df_full.groupby("transformed_location").apply(missing_votes)

In [76]:
def missing_reviews(df):
    df.Reviews = df.Reviews.fillna(df.Reviews.mean()) 
    return df
df_full.Reviews = df_full.Reviews.apply(lambda x:np.nan if x=='-' else float(x))
df_full =df_full.groupby("City").apply(missing_votes)
df_full.Reviews = df_full.Reviews.fillna(df_full.Reviews.mean())

In [77]:
df_full.drop(['Restaurant', 'Location','Cuisines'],inplace=True,axis=1)

In [78]:
df_test.shape

(2774, 8)

In [79]:
df_test = df_full[df_full.Delivery_Time.isnull()]

In [80]:
df_full.describe()

,Rating,Reviews,Votes,transformed_location,City,Count_cuisins,Cuisines_enc
count,13867.000000,13867.000000,13867.000000,13867.000000,13867.000000,13867.000000,13867.000000
mean,3.608694,120.893004,237.772249,16.909786,2.827865,2.345208,94.757842
std,0.376291,280.605025,496.364607,9.626750,2.690380,1.246501,67.385301
min,2.100000,1.000000,3.000000,0.000000,0.000000,1.000000,0.000000
25%,3.400000,10.000000,25.000000,8.000000,1.000000,1.000000,47.000000
50%,3.600000,48.000000,102.000000,17.000000,1.000000,2.000000,78.000000
75%,3.800000,120.893004,247.023529,27.000000,6.000000,3.000000,130.000000
max,4.900000,6504.000000,9054.000000,34.000000,7.000000,8.000000,468.000000


In [81]:
df_full.dropna(inplace = True)

In [82]:
df_full.head()

,Average_Cost,Delivery_Time,Minimum_Order,Rating,Reviews,Votes,Restaurant_name,transformed_location,City,Count_cuisins,Cuisines_enc
0,200,30 minutes,50,3.5,4.0,12.0,6321,10,7,5,193
1,100,30 minutes,50,3.5,4.0,11.0,2882,30,0,2,87
2,150,65 minutes,50,3.6,30.0,99.0,1595,19,6,3,93
3,250,30 minutes,99,3.7,95.0,176.0,5929,28,1,3,121
4,200,65 minutes,99,3.2,235.0,521.0,6123,26,0,2,112


In [83]:
# df_full.drop(['Restaurant', 'Location'],inplace=True,axis=1)

In [84]:
df_full.Average_Cost = df_full.Average_Cost.astype(int)
df_full.Minimum_Order = df_full.Minimum_Order.astype(int)
df_full.Restaurant_name = df_full.Restaurant_name.astype(int)
df_full.transformed_location = df_full.transformed_location.astype(int)

In [85]:
del_time_encode = LabelEncoder()
del_time_encode.fit(df_full.Delivery_Time)
df_full.Delivery_Time = del_time_encode.transform(df_full.Delivery_Time)

In [86]:
test = df_full.groupby("transformed_location").apply(lambda x: x.Delivery_Time.mean()).reset_index()

In [87]:
# df_full['Average_delivery_time'] = df_full['transformed_location'].map(dict(zip(test.transformed_location,test[0])))

In [88]:
# df_test['Average_delivery_time'] = df_test['transformed_location'].map(dict(zip(test.transformed_location,test[0])))

In [89]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [90]:
X = df_full.drop("Delivery_Time",axis = 1)
Y = df_full.Delivery_Time

In [91]:
# X = X[['Cuisines_enc','Rating','Reviews','Votes','Restaurant_name','transformed_location','City']]

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=56)

In [98]:
RC = RandomForestClassifier(n_estimators=1000,n_jobs=-1,max_leaf_nodes=1800,min_weight_fraction_leaf=0.0000000001)

In [99]:
RC.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=1800,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=1e-10, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [100]:
y_pred = RC.predict(X_test)

In [101]:
accuracy_score(y_test,y_pred)

0.8102748986029743

In [102]:
dict(zip(X_train,RC.feature_importances_))

{'Average_Cost': 0.06799794300674555,
 'Minimum_Order': 0.03793165719486229,
 'Rating': 0.10117379290766033,
 'Reviews': 0.18377926045588672,
 'Votes': 0.18424788617475368,
 'Restaurant_name': 0.14580588775251868,
 'transformed_location': 0.07372083103073974,
 'City': 0.04783029937451089,
 'Count_cuisins': 0.04275192125678394,
 'Cuisines_enc': 0.11476052084553814}

In [103]:
from sklearn.model_selection import cross_validate

In [137]:
cross_validate(RC,X,Y,error_score='accuracy',cv= 5)

/home/sidhartha/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


{'fit_time': array([2.99743271, 3.12940097, 3.03904724, 3.84422088, 3.86201572]),
 'score_time': array([0.30877447, 0.30667162, 0.30694771, 0.30542016, 0.30443645]),
 'test_score': array([0.78963964, 0.78423423, 0.78323569, 0.79215509, 0.79873646])}

In [51]:
# # from sklearn.model_selection import GridSearchCV
# params ={'bootstrap': [True, False],
#  'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#  'max_features': ['auto', 'sqrt',4,6,8],
#  'min_samples_leaf': [1, 2, 4],
#  'min_samples_split': [2, 5, 10],
#  'n_estimators': [800, 1000, 1200, 1400, 1600, 1800, 2000]}
# gcv = GridSearchCV(RC,param_grid=params,scoring='accuracy',cv=5,n_jobs=-1)
# gcv.fit(X,Y)

In [156]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators = 10,learning_rate=0.0001,num_leaves=None,min_child_weight=None)

lgbm.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.0001, max_depth=-1,
               min_child_samples=20, min_child_weight=None, min_split_gain=0.0,
               n_estimators=10, n_jobs=-1, num_leaves=None, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [157]:
y_pred = lgbm.predict(X_test)

In [158]:
accuracy_score(y_test,y_pred)

0.6396396396396397

In [104]:
# Voting classifier

In [105]:
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import RidgeClassifier


In [121]:
dst = DecisionTreeClassifier()
nb = MultinomialNB()
adab = AdaBoostClassifier()
Ext = ExtraTreesClassifier()
knn = KNeighborsClassifier()
xgb = XGBClassifier(max_depth=50,n_estimators=1000)
lgbm = LGBMClassifier()
rdgc = RidgeClassifier()


In [127]:
vc = VotingClassifier(estimators= [('clf1',dst),('clf3',adab),('clf4',Ext),
                                   ('clf6',xgb),('clf7',lgbm),('clf8',RC)])

In [128]:
vc.fit(X_train,y_train)

/home/sidhartha/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


VotingClassifier(estimators=[('clf1',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=None,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort=False,
                                                     random_state=None,
                                                     splitter='best')),
    

In [129]:
y_pred = vc.predict(X_test)
accuracy_score(y_test,y_pred)

0.8039657503379901

In [130]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [131]:
n_HP_points_to_test = 100

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='accuracy',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [133]:
gs.fit(X_train, y_train)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed: 58.3min finished


Best score reached: 0.7543385170160019 with params: {'colsample_bytree': 0.952164731370897, 'min_child_samples': 111, 'min_child_weight': 0.01, 'num_leaves': 38, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.3029313662262354} 


In [143]:
############### Start: Randomized Search CV ##################################

# Look at parameters used by our current forest
# from sklearn.ensemble import RandomForestRegressor

rf = RandomForestClassifier()

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

from sklearn.model_selection import RandomizedSearchCV

# Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'n_jobs':[8]
              }

pprint(random_grid)


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}
{'max_depth': [5, 10, 15, 20, 25, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 5, 10],
 'min_samples_split': [2, 5, 10, 15, 100],
 'n_estimators': [100,
                  200,
                  300,
                  400,
                  500,
                  600,
                  700,
                  800,
                  900,
                  1000,
                  1100,
                  1200],
 'n_jobs': [8]}


In [144]:
from sklearn.model_selection import RandomizedSearchCV

In [145]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = 1)

In [146]:
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_jobs=8, n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_jobs=8, n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   2.8s
[CV] n_jobs=8, n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s


[CV]  n_jobs=8, n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   0.7s
[CV] n_jobs=8, n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5 
[CV]  n_jobs=8, n_estimators=400, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=5, total=   0.6s
[CV] n_jobs=8, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=20 
[CV]  n_jobs=8, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=20, total=   0.7s
[CV] n_jobs=8, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=20 
[CV]  n_jobs=8, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=20, total=   0.7s
[CV] n_jobs=8, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=20 
[CV]  n_jobs=8, n_estimators=200, min_samples_split=5, min_samples_leaf=1, max

...

[CV]  n_jobs=8, n_estimators=700, min_samples_split=10, min_samples_leaf=10, max_features=auto, max_depth=25, total=   1.7s
[CV] n_jobs=8, n_estimators=700, min_samples_split=10, min_samples_leaf=10, max_features=auto, max_depth=25 
[CV]  n_jobs=8, n_estimators=700, min_samples_split=10, min_samples_leaf=10, max_features=auto, max_depth=25, total=   1.8s
[CV] n_jobs=8, n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=30 
[CV]  n_jobs=8, n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=30, total=   2.5s
[CV] n_jobs=8, n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=30 
[CV]  n_jobs=8, n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=30, total=   2.3s
[CV] n_jobs=8, n_estimators=1000, min_samples_split=15, min_samples_leaf=10, max_features=sqrt, max_depth=30 
[CV]  n_jobs=8, n_estimators=1000, min_samples_split=15, min_sa

[CV]  n_jobs=8, n_estimators=900, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=30, total=   2.1s
[CV] n_jobs=8, n_estimators=900, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=30 
[CV]  n_jobs=8, n_estimators=900, min_samples_split=100, min_samples_leaf=10, max_features=sqrt, max_depth=30, total=   2.0s
[CV] n_jobs=8, n_estimators=200, min_samples_split=100, min_samples_leaf=2, max_features=sqrt, max_depth=30 
[CV]  n_jobs=8, n_estimators=200, min_samples_split=100, min_samples_leaf=2, max_features=sqrt, max_depth=30, total=   0.6s
[CV] n_jobs=8, n_estimators=200, min_samples_split=100, min_samples_leaf=2, max_features=sqrt, max_depth=30 
[CV]  n_jobs=8, n_estimators=200, min_samples_split=100, min_samples_leaf=2, max_features=sqrt, max_depth=30, total=   0.6s
[CV] n_jobs=8, n_estimators=200, min_samples_split=100, min_samples_leaf=2, max_features=sqrt, max_depth=30 
[CV]  n_jobs=8, n_estimators=200, min_samples_split=100, min_samp

[CV]  n_jobs=8, n_estimators=300, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=10, total=   0.8s
[CV] n_jobs=8, n_estimators=300, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=10 
[CV]  n_jobs=8, n_estimators=300, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=10, total=   0.9s
[CV] n_jobs=8, n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=15 
[CV]  n_jobs=8, n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=15, total=   1.5s
[CV] n_jobs=8, n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=15 
[CV]  n_jobs=8, n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=15, total=   1.6s
[CV] n_jobs=8, n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=15 
[CV]  n_jobs=8, n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_featu

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  8.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [150]:
accuracy_score(y_test,y_pred)

0.8053177106804867

In [149]:
y_pred = rf_random.best_estimator_.predict(X_test)

In [165]:
y_pred = RC.predict(df_test.drop("Delivery_Time",axis=1)[X.columns])

In [166]:
df_submit = pd.DataFrame({"Delivery_Time":del_time_encode.inverse_transform(y_pred)})

In [167]:
df_submit

,Delivery_Time
0,30 minutes
1,30 minutes
2,30 minutes
3,30 minutes
4,30 minutes
...,...
2769,30 minutes
2770,30 minutes
2771,45 minutes
2772,30 minutes


In [168]:
df_submit.to_excel("submit.xlsx",index = False)